In [1]:
import time
import torch
import numpy as np
import torch.nn as nn
from torch import optim
from utils import fidelity
from QudiTop.gates import *
from numpy.linalg import norm
from QudiTop.circuit import Circuit
from QudiTop.global_var import DTYPE
from scipy.stats import unitary_group
from QudiTop.expectation import Expectation

np.set_printoptions(linewidth=250)
torch.set_printoptions(linewidth=250)

In [2]:
circ = Circuit(3, 2) + RY(3, [0, 1], 'a').on(0, 1)
circ.matrix()
psi0 = np.random.rand(9) + 1j * np.random.rand(9)
print(psi0)
circ.set_init_qs(psi0)
print(circ.get_qs())
circ.reset()
print(circ.get_qs())

[0.45331642+0.79747202j 0.42448859+0.07443983j 0.19765934+0.00923311j 0.15511285+0.0483393j  0.03690061+0.51738229j 0.40335817+0.32363244j 0.56255417+0.3520578j  0.47538241+0.02977677j 0.30546885+0.40718751j]
[0.45331642+0.79747202j 0.42448859+0.07443983j 0.19765934+0.00923311j 0.15511285+0.0483393j  0.03690061+0.51738229j 0.40335817+0.32363244j 0.56255417+0.3520578j  0.47538241+0.02977677j 0.30546885+0.40718751j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [3]:
def ZYZ(d, name, obj, with_phase: bool = False):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit(d, 1)
    index = [[1, 2], [0, 1], [1, 2]]
    for i, ind in enumerate(index):
        str_pr = f'{"".join(str(i) for i in ind)}_{i}'
        circ += RZ(d, ind, f'{name}RZ{str_pr}').on(obj)
        circ += RY(d, ind, f'{name}RY{str_pr}').on(obj)
        circ += RZ(d, ind, f'{name}Rz{str_pr}').on(obj)
        if with_phase:
            circ += GP(d, f'{name}phase_{i}').on(obj)
    return circ


def Cd(d, name, obj, ctrl, state):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit(d, 2)
    circ += RZ(d, [0, 1], f'{name}RZ01').on(obj, ctrl, state)
    circ += RZ(d, [0, 2], f'{name}RZ02').on(obj, ctrl, state)
    circ += GP(d, f'{name}phase').on(obj, ctrl, state)
    return circ


def qutrit_ansatz(gate: UMG, with_phase: bool = True):
    d = gate.dim
    obj = gate.obj_qudits
    name = f'{gate.name}_'
    circ = Circuit(d, 2)
    if len(obj) == 1:
        circ += ZYZ(d, f'{name}', obj[0])
    elif len(obj) == 2:
        circ += ZYZ(d, f'{name}U1_', obj[0])
        circ += Cd(d, f'{name}Cd1_', obj[0], obj[1], 1)
        circ += ZYZ(d, f'{name}U2_', obj[0])
        circ += Cd(d, f'{name}Cd2_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U3_', obj[0])
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 2)
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 1)
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U4_', obj[0])
        circ += Cd(d, f'{name}Cd3_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U5_', obj[0])
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 2)
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 1)
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U6_', obj[0])
        circ += Cd(d, f'{name}Cd4_', obj[0], obj[1], 0)
        circ += ZYZ(d, f'{name}U7_', obj[0])
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 2)
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 1)
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U8_', obj[0])
        circ += Cd(d, f'{name}Cd5_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U9_', obj[0])
    else:
        raise ValueError('Only works when nq <= 2')
    if with_phase:
        circ += [GP(d, 'phase').on(i) for i in obj]
    return circ

In [4]:
d, nq = 3, 2
circ = Circuit(d, nq)
ansatz = Circuit(d, nq)
mat = unitary_group.rvs(d**nq, random_state=42)
obj = list(range(nq))
gate = UMG(d, mat, name=f'mat').on(obj)
circ += gate
ansatz += qutrit_ansatz(gate, True)
for i, g in enumerate(ansatz.gates):
    print('{:3d}'.format(i), g)

pr = ansatz.get_parameters()
g_num = len(ansatz.gates)
p_num = len(pr)
print('Number of qudits: %d' % nq)
print('Number of params: %d' % p_num)
print('Number of gates: %d' % g_num)

psi = circ.get_qs()
rho = np.outer(psi, psi.conj())
print('Hamiltonian Dimension:', rho.shape)
Ham = [(1, UMG(d, rho).on(obj))]
expect = Expectation(Ham)

  0 RZ(3 [1 2] mat_U1_RZ12_0|0)
  1 RY(3 [1 2] mat_U1_RY12_0|0)
  2 RZ(3 [1 2] mat_U1_Rz12_0|0)
  3 RZ(3 [0 1] mat_U1_RZ01_1|0)
  4 RY(3 [0 1] mat_U1_RY01_1|0)
  5 RZ(3 [0 1] mat_U1_Rz01_1|0)
  6 RZ(3 [1 2] mat_U1_RZ12_2|0)
  7 RY(3 [1 2] mat_U1_RY12_2|0)
  8 RZ(3 [1 2] mat_U1_Rz12_2|0)
  9 RZ(3 [0 1] mat_Cd1_RZ01|0 <-: 1 - 1)
 10 RZ(3 [0 2] mat_Cd1_RZ02|0 <-: 1 - 1)
 11 GP(3 mat_Cd1_phase|0 <-: 1 - 1)
 12 RZ(3 [1 2] mat_U2_RZ12_0|0)
 13 RY(3 [1 2] mat_U2_RY12_0|0)
 14 RZ(3 [1 2] mat_U2_Rz12_0|0)
 15 RZ(3 [0 1] mat_U2_RZ01_1|0)
 16 RY(3 [0 1] mat_U2_RY01_1|0)
 17 RZ(3 [0 1] mat_U2_Rz01_1|0)
 18 RZ(3 [1 2] mat_U2_RZ12_2|0)
 19 RY(3 [1 2] mat_U2_RY12_2|0)
 20 RZ(3 [1 2] mat_U2_Rz12_2|0)
 21 RZ(3 [0 1] mat_Cd2_RZ01|0 <-: 1 - 2)
 22 RZ(3 [0 2] mat_Cd2_RZ02|0 <-: 1 - 2)
 23 GP(3 mat_Cd2_phase|0 <-: 1 - 2)
 24 RZ(3 [1 2] mat_U3_RZ12_0|0)
 25 RY(3 [1 2] mat_U3_RY12_0|0)
 26 RZ(3 [1 2] mat_U3_Rz12_0|0)
 27 RZ(3 [0 1] mat_U3_RZ01_1|0)
 28 RY(3 [0 1] mat_U3_RY01_1|0)
 29 RZ(3 [0 1] mat_U3_Rz01_1

In [5]:
start = time.perf_counter()
p0 = np.random.uniform(-1, 1, p_num)
target = torch.tensor([1], dtype=DTYPE)
ansatz.assign_ansatz_parameters(dict(zip(pr, p0)))
optimizer = optim.Adam(ansatz.parameters(), lr=1e-2)
for i in range(500):
    out = expect(ansatz())
    loss = nn.L1Loss()(out, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        t = time.perf_counter() - start
        print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))
    if loss < 1e-8:
        break
t = time.perf_counter() - start
print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))

pr_res = ansatz.get_parameters()
psi_res = ansatz.get_qs()
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res))
print('psi fidelity: %.20f' % fidelity(psi, psi_res)**2)

end = time.perf_counter()
print('Runtime: %f' % (end - start))

Loss: 0.836089819837739, Fidelity: 0.163910180162261,   0, 0.5831
Loss: 0.577500493360390, Fidelity: 0.422499506639610,  10, 1.2046
Loss: 0.554707322178237, Fidelity: 0.445292677821763,  20, 1.8303
Loss: 0.508502500987896, Fidelity: 0.491497499012104,  30, 2.4430
Loss: 0.462370049918449, Fidelity: 0.537629950081551,  40, 3.0529
Loss: 0.400930816129370, Fidelity: 0.599069183870630,  50, 3.6331
Loss: 0.330917382992350, Fidelity: 0.669082617007650,  60, 4.2051
Loss: 0.263188447258769, Fidelity: 0.736811552741231,  70, 4.8094
Loss: 0.200586545379498, Fidelity: 0.799413454620502,  80, 5.4471
Loss: 0.146871500757218, Fidelity: 0.853128499242782,  90, 6.1172
Loss: 0.103143202435203, Fidelity: 0.896856797564797, 100, 6.7661
Loss: 0.069069131503439, Fidelity: 0.930930868496561, 110, 7.4174
Loss: 0.044724859896496, Fidelity: 0.955275140103504, 120, 8.0400
Loss: 0.028663746503521, Fidelity: 0.971336253496479, 130, 8.6550
Loss: 0.018085470759607, Fidelity: 0.981914529240393, 140, 9.2865
Loss: 0.01